<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/lesson3/NLP_intro_hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 3. ДЗ Embedding word2vec fasttext
## Задача поиск похожих по эмбеддингам
на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы

In [42]:
!pip install pymorphy2 -q

In [43]:
!pip install annoy -q

In [44]:
! pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
!pip install stop_words -q

In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange, tqdm_notebook

from corus import load_lenta

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

In [47]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz -q

In [48]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [49]:
df = pd.DataFrame(columns=['title', 'text'])

In [50]:
%%time

c = 0

for article in tqdm_notebook(records):
    df = df.append({'title': article.title, 'text': article.text}, ignore_index=True)
    c += 1
    if c > 100000:
        break

<timed exec>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

CPU times: user 10min 53s, sys: 5.5 s, total: 10min 58s
Wall time: 11min 14s


In [51]:
df.shape
df.to_csv('lenta_data.csv')
df = pd.read_csv('lenta_data.csv')

In [52]:
assert True

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [53]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [54]:
df = df.dropna(subset=['title'])

# title

Попробуем сгенерировать близкие по названию статьи.

In [55]:
%%time

c = 0
title_sentences = []

for line in tqdm_notebook(df['title']):
    spls = preprocess_txt(line)
    title_sentences.append(spls)
    c += 1
    if c > 100000:
        break

<timed exec>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100001 [00:00<?, ?it/s]

CPU times: user 7min 59s, sys: 2.92 s, total: 8min 2s
Wall time: 8min 9s


In [62]:
len(title_sentences)

100001

In [64]:
%%time

modelW2V_title = Word2Vec(sentences=title_sentences, size=300, window=5, min_count=3)

CPU times: user 21.6 s, sys: 223 ms, total: 21.8 s
Wall time: 16.2 s


In [65]:
%%time

modelFT_title = FastText(sentences=title_sentences, size=300, min_count=3, window=5, workers=8)

CPU times: user 1min 25s, sys: 2.06 s, total: 1min 27s
Wall time: 57.8 s


In [66]:
%%time

w2v_index_title = annoy.AnnoyIndex(300 ,'angular')
ft_index_title = annoy.AnnoyIndex(300 ,'angular')

index_map_title = {}
counter = 0

for title in tqdm_notebook(df['title']):
    n_w2v = 0
    n_ft = 0
    index_map_title[counter] = title
    article_morphs = preprocess_txt(title)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in article_morphs:
        if word in modelW2V_title.wv:
            vector_w2v += modelW2V_title.wv[word]
            n_w2v += 1
        if word in modelFT_title.wv:
            vector_ft += modelFT_title.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index_title.add_item(counter, vector_w2v)
    ft_index_title.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index_title.build(10)
ft_index_title.build(10)

<timed exec>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100001 [00:00<?, ?it/s]

CPU times: user 9min 13s, sys: 5.71 s, total: 9min 19s
Wall time: 9min 25s


True

In [67]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [68]:
TEXT = "Чемпион России по футболу"

In [69]:
get_response(TEXT, w2v_index_title, modelW2V_title, index_map_title)

['На чемпионат Европы по\xa0ММА поедут восемь чемпионов России',
 '«Спартак» упустил победу в\xa0Кубке России',
 'Олимпийский чемпион порадовался невызову в\xa0сборную России по\xa0хоккею',
 'Женская сборная России по\xa0футболу вылетела с\xa0чемпионата Европы',
 'Определился соперник сборной России по\xa0полуфиналу чемпионата Европы по\xa0волейболу']

In [70]:
get_response(TEXT, ft_index_title, modelFT_title, index_map_title)

['Чемпион России захотел футболиста «Барселоны»',
 'В Германии задумались о\xa0бойкоте чемпионата мира по\xa0футболу в\xa0России',
 'На чемпионат Европы по\xa0ММА поедут восемь чемпионов России',
 '«Спартак» обыграл ЦСКА в\xa0чемпионате России',
 'Олимпийский чемпион из\xa0Норвегии поболеет за\xa0Россию на\xa0чемпионате мира по\xa0футболу']

В принципе если задавать вопрос из корпуса, то получаем достаточно близкий результат.

# Text

In [ ]:
%%time

c = 0
text_sentences = []

for line in tqdm_notebook(df['text']):
    spls = preprocess_txt(line)
    text_sentences.append(spls)
    c += 1
    if c > 100000:
        break

По текстам обучается очень долго, поэтому не проводил проверку.

In [ ]:
%%time

modelW2V_text = Word2Vec(sentences=text_sentences, size=300, window=5, min_count=3)

In [ ]:
%%time
# vector_size
modelFT_text = FastText(sentences=text_sentences, size=300, min_count=3, window=5, workers=8)

In [ ]:
%%time

w2v_index_text = annoy.AnnoyIndex(300 ,'angular')
ft_index_text = annoy.AnnoyIndex(300 ,'angular')

index_map_text = {}
counter = 0

for text in tqdm_notebook(df['text']):
    n_w2v = 0
    n_ft = 0
    index_map_text[counter] = text
    article_morphs = preprocess_txt(text)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in article_morphs:
        if word in modelW2V_text.wv:
            vector_w2v += modelW2V_text.wv[word]
            n_w2v += 1
        if word in modelFT_text.wv:
            vector_ft += modelFT_text.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index_text.add_item(counter, vector_w2v)
    ft_index_text.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index_text.build(10)
ft_index_text.build(10)

In [ ]:
TEXT = "Чемпион России по футболу"

In [ ]:
# index_map_text = index_map

get_response(TEXT, w2v_index_text, modelW2V_text, index_map_text)

In [ ]:
get_response(TEXT, ft_index_text, modelFT_text, index_map_text)